## SHAP Values

### A brief introduction to shapley values

SHAP values are generated through the [SHAP library](https://link.springer.com/article/10.1007/s10115-013-0679-x) and are approximations of [Shapley Values](https://en.wikipedia.org/wiki/Shapley_value), a concept derived from game-theory. 

Rsmexplain by default uses the [Sampling](https://shap.readthedocs.io/en/latest/generated/shap.explainers.Sampling.html#shap.explainers.Sampling) explainer model, which computes shap values 

Shap values are additive representations of a feature's impact on a model decision. The sum of all shap values and the base value for a prediction should yield the actual model output.

### How to read shap values.

### Things to consider

### An overview over your shap values